# Program II: Benchmarking Insertion and Selection Sorts
### By: Drew Mattson, Ian Czerkis, Akashdeep Gill
## Introduction

In [19]:
import copy
import time
import random
import matplotlib.pyplot as plt
import unittest

# Sorting Algorithms
## Insertion Sort

In [20]:
def insertion_sort(lst):
    size = len(lst)
    # iterate from index 1 to array length
    for j in range(1, size):
        key = lst[j]
        i = j - 1
        # shift elements to appropriate spots if necessary
        while i >= 0 and lst[i] > key:
            lst[i+1] = lst[i]
            i = i - 1
        # place key in its appropriate sorted position
        lst[i+1] = key


## Selection Sort

In [21]:
def selection_sort(lst):
    size = len(lst)
    for ind in range(size):
        min_index = ind
 
        for j in range(ind + 1, size):
            # select the minimum element in every iteration
            if lst[j] < lst[min_index]:
                min_index = j
         # swap the elements to sort the array
        (lst[ind], lst[min_index]) = (lst[min_index], lst[ind])

## Correctness Testing

In [22]:
original_tests_cases = [
    ([3, 2, 1], [1, 2, 3]),
    ([1, 2, 3], [1, 2, 3]),
    ([1, 8, 7, 9, 2, 4, 10, 5, 6, 3], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
    ([492, 215, 761, 38, 904, 127, 589, 333, 674, 51], [38, 51, 127, 215, 333, 492, 589, 674, 761, 904]),
    ([724, 138, 502, 891, 267, 605, 49, 803, 416, 175], [49, 138, 175, 267, 416, 502, 605, 724, 803, 891]),
    ([0.587, 0.231, 0.895, 0.412, 0.674, 0.138, 0.759, 0.523, 0.946, 0.301], [0.138, 0.231, 0.301, 0.412, 0.523, 0.587, 0.674, 0.759, 0.895, 0.946]),
    ([-55, 72, -18, 41, -93, 7, 64, -29, 15, -84], [-93, -84, -55, -29, -18, 7, 15, 41, 64, 72])
]

def get_deep_copy_of_test_cases():
    return copy.deepcopy(original_tests_cases)

class TestAlgorithms(unittest.TestCase):
    def test_selection_sort(self):
        test_cases = get_deep_copy_of_test_cases()
        for input_lst, expected_output in test_cases:
            with self.subTest(input_lst=input_lst):
                selection_sort(input_lst)
                self.assertEqual(input_lst, expected_output)

    def test_insertion_sort(self):
            test_cases = get_deep_copy_of_test_cases()
            for input_lst, expected_output in test_cases:
                with self.subTest(input_lst=input_lst):
                    insertion_sort(input_lst)
                    self.assertEqual(input_lst, expected_output)

unittest.main(argv=['first-arg-is-ignored'], exit=False)


FF
FAIL: test_insertion_sort (__main__.TestAlgorithms.test_insertion_sort) (input_lst=[1, 2, 3])
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\1251190474.py", line 27, in test_insertion_sort
    self.assertEqual(input_lst, expected_output)
AssertionError: Lists differ: [1, 2, 3] != [1, 2, 2]

First differing element 2:
3
2

- [1, 2, 3]
?        ^

+ [1, 2, 2]
?        ^


FAIL: test_selection_sort (__main__.TestAlgorithms.test_selection_sort) (input_lst=[1, 2, 3])
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\1251190474.py", line 20, in test_selection_sort
    self.assertEqual(input_lst, expected_output)
AssertionError: Lists differ: [1, 2, 3] != [1, 2, 2]

First differing element 2:
3
2

- [1, 2, 3]
?        ^

+ [1, 2, 2]
?        ^


---------------

# Benchmarking
## Set Up Benchmarks

In [23]:
algorithms = [selection_sort, insertion_sort]
# benchmark_sizes = [10, 100, 1000, 10000]
benchmark_sizes = [10, 100, 1000, 10000, 100000]
benchmark_cases = ['sorted', 'random', 'reversed']

def generate_random_list(size, case):
    if case == 'random':
        return random.sample(range(1, size + 1), size)
    elif case == 'sorted':
        return list(range(1, size + 1))
    elif case == 'reversed':
        return list(range(size, 0, -1))

def benchmark(sorting_algorithm, lst):
    # set up
    copy_lst = copy.deepcopy(lst)
    start_time = time.perf_counter()
    # sort
    sorting_algorithm(copy_lst)
    # tear down
    end_time = time.perf_counter()
    return end_time - start_time

def benchmark_algorithms():
    results = {}
    for size in benchmark_sizes:
        for case in benchmark_cases:
            lst = generate_random_list(size, case)
            for algorithm in algorithms:
                key = (algorithm.__name__, size, case)
                # print(f'Benchmarking {key}')
                results[key] = benchmark(algorithm, lst)
    
    return results

def plot_benchmark():
    results = benchmark_algorithms()

    insertion_sort_colors = ['steelblue', 'cornflowerblue', 'royalblue']
    selection_sort_colors = ['lightcoral', 'indianred', 'darkred']

    for i, case in enumerate(benchmark_cases):
        # Plot insertion_sort
        insertion_sort_data = [(size, results[('insertion_sort', size, case)]) for size in benchmark_sizes]
        plt.plot([size for size, _ in insertion_sort_data], [sort_time for _, sort_time in insertion_sort_data],
                 label=f'insertion_sort - {case}', color=insertion_sort_colors[i])

        # Plot selection_sort
        selection_sort_data = [(size, results[('selection_sort', size, case)]) for size in benchmark_sizes]
        plt.plot([size for size, _ in selection_sort_data], [sort_time for _, sort_time in selection_sort_data],
                 label=f'selection_sort - {case}', color=selection_sort_colors[i])

    # Add labels and legend
    plt.xlabel('List Size')
    plt.ylabel('Algorithm Execution Time')
    plt.title('Algorithm Benchmarking')
    plt.legend()

    # Show the plot
    plt.show()

## Run Benchmark

In [24]:
plot_benchmark()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mattsona\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\336384413.py", line 1, in <module>
    plot_benchmark()
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\2148225849.py", line 37, in plot_benchmark
    results = benchmark_algorithms()
              ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\2148225849.py", line 32, in benchmark_algorithms
    results[key] = benchmark(algorithm, lst)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\2148225849.py", line 19, in benchmark
    sorting_algorithm(copy_lst)
  File "C:\Users\mattsona\AppData\Local\Temp\ipykernel_688\3373129182.py", line -1, in selection_sort
KeyboardInterrupt

During handling of the above e

# Reflection
## Theoretical vs Measured Algorithm Performance
| Algorithm          | Case     | Theoretical Run Time   | Estimated Run Time   |
|-------------------- |----------|-------------------------|----------------------|
| Insertion Sort      | Best     | O(n)                    | O(n)                 |
| Insertion Sort      | Average  | O(n^2)                  | O(n^2)               |
| Insertion Sort      | Worst    | O(n^2)                  | O(n^2)               |
| Selection Sort      | Best     | O(n^2)                  | O(n^2)               |
| Selection Sort      | Average  | O(n^2)                  | O(n^2)               |
| Selection Sort      | Worst    | O(n^2)                  | O(n^2)               |

##Do they behave as expected


## Case by Case Analysis
# Algorithm Performance:

For best and average cases, Insertion Sort is expected to perform better as it has linear time complexity in the best case.
In the worst case, both algorithms have the same quadratic time complexity, so their performance should be comparable.

# Inner Loop Consideration:

Insertion Sort is generally more efficient in practice for small datasets and already partially sorted data due to its adaptive nature.Selection Sort's inner loop involves finding the minimum element and swapping, which is inherently less efficient than the comparisons in Insertion Sort.

# Practical Usage:
In practice, the choice between insertion sort and selection sort would depend on the specific requirements and characteristics of the input data.

Use Insertion Sort when:

- The list is expected to be partially sorted or has a high likelihood of being already sorted.
- The overhead of the algorithm is acceptable for small to medium-sized lists.
- Stability in sorting is crucial (insertion sort is stable).

Use Selection Sort when:

Memory usage needs to be minimized as it performs a constant number of swaps.
- The list is relatively small, and simplicity is preferred over efficiency.
- The stability of sorting is not a primary concern.

Ultimately, the choice between insertion sort and selection sort would be application-dependent, considering factors such as input data characteristics, size of the data, and specific performance requirements.

## Overall Analysis
// TODO